In [ ]:
#default_exp trainers
#export 
import pytorch_lightning as lit

def cross_validate(ldhdata, k=3):
    for i in range(num_folds):
        ldhdata.current_split = i
        train_dl = ldhdata.train_dataloader()
        val_dl = ldhdata.val_dataloader()
        trainer = lit.Trainer()

In [ ]:
#hide
from nbdev.export import notebook2script; 
notebook2script("Trainers.ipynb");


Converted Trainers.ipynb.
